# Microcircuits and Descretes

In [1]:
cd C:\Users\lselmi\cygwin64\home\lselmi\anaconda\altertech\datasets

C:\Users\lselmi\cygwin64\home\lselmi\anaconda\altertech\datasets


In [48]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas import Series, DataFrame
%matplotlib inline
import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')
plt.style.use('seaborn-whitegrid')
pd.__version__

'0.22.0'

### Read the data

In [85]:
sales = pd.read_csv('PurchaseData_20180319.csv')

# Microcircuits Data Analysis
In order to predict the price (POP_UnitPrice) and the delivery time (PO_Date) of microcircuits we have received from ATN, with the data, a list of features that should be relevant for the task at hand and a set of rules. The relevant features are a subset of the fields in the data set.

1. Component number (ComponentNumber_MAT_Flight)
2. Specification name (SpecificationName)
3. Family path (FamilyPath_Flight)
4. Style (Style_Flight)
5. Quality level (QLevel_Flight)
6. Package class (PACKAGECLASS)
7. Package (PACKAGE)
8. Finish (FINISH)
9. Radiation level (TID_HDR_N)
10. Quality Value Name (QualityValueName)
11. Manufacturer (MnfrDoeeetName)
12. Quantity (POP_Qty)
13. Date of purchase (PO_Date)
14. Unit price (POP_UnitPrice)
15. Date of delivery (POP_DeliveryDate)

The assumption is that the data and the rules should allow us to predict the price and delivery time of a microcircuit whether there are records about that specific microcircuit in the sample data set or not. The first 12 paramenters are called features or predictors while the unit price and the date of delivery are called targets. A client, requesting a prediction about the price of a component, will send in the request the predictors that will allow the server to 

1. Identify the component (component number, specification name, family path)
2. Determine the quality characteristics of the component (style, package, package class, finish, radiation level, quality value)
3. Use other information that might impact the price (manufacturer, quantity, date of purchase) 

In order to make a prediction of the price of a component, the algorithm looks into the data to find records about that same component and return the unit price or an average value. In case no records are available about that component number, it looks for records with the same specification name and applies some rules to make a prediction for the price. 

The component number and the specification name encode, among other information, the specific family of the component, e.g. whether it is an operational amplifier or an analog to digital converter. When there are no records with the same component number or specification name, the algorithm looks for records about similar components and apply some rules to make a prediction. So the next step in this case is to look for records with the same family path or the same most specific name in the family path. 


## Data preparation for microcircuits
Before implementing the algorithm to predict unit price and delivery time for a microcircuit, we have to extract the records from the sale orders data set and apply the following transformation

1. Filter out the records about services (aka "charges") 
2. Select the records about microcircuits (family root -> microcircuits)
3. Extract the most specific family of the component from the hierarchy (family path)
4. Transform all the prices in euro
5. Update the all the unit prices applying an increase of 5 % per year (using the compund interest formula) 

After the data is prepared we can implement the algorithms for microcircuits 

1. Price prediction
2. Delivery time prediction


####  1. Filter out records about services

In [86]:
sales = sales[sales['PRICE LABEL'] == 'MATERIAL UNIT PRICE']
sales.index.size

18508

#### 2. Select the records about microcircuits

In [87]:
microcircuits_records = sales[sales['FamilyRoot'] == 'Microcircuits']
num_microcircuits_records = microcircuits_records.index.size
print("Number of records for microcircuits: " + str(num_microcircuits_records))

Number of records for microcircuits: 3041


#### 3. Extract the family root and leaf 

In [88]:
microcircuits_records['family_leaf'] = [family.split("/")[len(family.split("/")) - 1] for family in microcircuits_records['FamilyPath_Flight'] ]
microcircuits_records['family_root'] = [family.split("/")[0] for family in microcircuits_records['FamilyPath_Flight'] ]
print("Family root: " + microcircuits_records['family_root'][0] + ", Family leaf: " + microcircuits_records['family_leaf'][0])

Family root: Microcircuits, Family leaf: Operational Amplifier


#### 4. Transform all the unit prices in US dollars to euros

In [89]:
microcircuits_records['price_euros'] = microcircuits_records['POP_UnitPrice_CU'] * microcircuits_records['PO_Change'] * (microcircuits_records['PO_Currency'] == 'USD')
microcircuits_records['price_euros'] += microcircuits_records['POP_UnitPrice_CU'] * (microcircuits_records['PO_Currency'] == 'EUR')

In [90]:
#microcircuits_records['price_euros_simple'] = [price * microcircuits_records['PO_Change'] for price in microcircuits_records['POP_UnitPrice_CU']]

#### 5. Update the unit prices
The date of purchase is used to compute the adjusted price (AP) from the unit price (P) in each record using the formula 

> AP = P*(1 + %)^Y

where % is the increase in price per year, e.g. 5 %, Y is the number of years since the date of purchase in the record.

In [117]:
# Change the type of PO_Date from int64 to str
years_str = pd.Series(microcircuits_records['PO_Date']).astype('str')
# Extract the 1st 4 digits
years_str = [year_str[0:4] for year_str in years_str]
# Change back from str to int and compute the number of years from the purchase date to 2018
years = [2018 - int(year_str) for year_str in years_str]
microcircuits_records['years'] = years
microcircuits_records['adjusted_price'] = microcircuits_records['price_euros'] * np.power(1 + 0.05, microcircuits_records['years'])
print("Price in " + str(2018 - microcircuits_records['years'][0]) + ": " + str(microcircuits_records['price_euros'][0]) + ". Adjusted price: " + str(microcircuits_records['adjusted_price'][0]))

Price in 2013: 9.85. Adjusted price: 12.571373390625004


## Price prediction for microcircuits
The client will send all the 12 predictors, each mapped to a field in the dataset

1. Component number
2. Specification name
3. Family path
4. Style 
5. Quality level
6. Package class
7. Package
8. Finish 
9. Radiation level
10. Quality Value Name
11. Manufacturer
12. Quantity

In order to make a prediction the algorithm must find some records about the same component or  a similar one in the sale orders. The following three scenarios might happen

1. The component number in the request matches with a component number in the dataset
2. The component number in the request does not match with any in the dataset but the specification name matches
3. Neither the component number nor the specification name in the request matches with a record in the sale orders

### Scenario 1
The component number in the request matches with a component number in the datase.

In this scenario the only parameters to use to predict the price are

- Manufacturer
- Date of purchase
- Quantity

The manufacturer is used to select the records with the same manufacturer. If the manufacturer is different the records with the different manufacturer will be used. 

Compute the average adjusted price for the same quantity in the sale orders and the standard deviation. This step can be performed in the data preparation phase.

If more than one records are availabe with different quantities use a linear interpolation average adjusted prices to find the average adjusted price for the quantity requested. If only one record is available, returns the adjusted price.


### Scenario 2
The component number in the request does not match with any in the dataset but the specification name matches.

In this scenario the algorithm must select the records that are about a similar component using the specification name and the family path.

It must also filter the records, about the same specification name and family path

### Scenario 3
Neither the component number nor the specification name in the request matches with a record in the sale orders